In [23]:

@elapsed using DataFrames



1.047909333

In [24]:

@elapsed using CSV

0.002354424

In [25]:
to_add = Dict()
to_add["block size"] = 6
to_add["new0"] = "new"

tbl = CSV.read("../data2.csv")
display(typeof(tbl))
display(tbl)

DataFrame

,datetime,block size,comment,time
,String,Int64,String,Float64
1,2019-04-26 12:26:10,16,computed,126.29
2,2019-04-26 12:28:16,32,computed,111.373
3,2019-04-26 12:30:08,64,computed,101.213
4,2019-04-26 12:31:49,16,memory and file,9.427
5,2019-04-26 12:31:58,32,memory and file,10.253
6,2019-04-26 12:32:09,64,memory and file,10.16


In [26]:
tbl = DataFrame(tbl)

,datetime,block size,comment,time
,String,Int64,String,Float64
1,2019-04-26 12:26:10,16,computed,126.29
2,2019-04-26 12:28:16,32,computed,111.373
3,2019-04-26 12:30:08,64,computed,101.213
4,2019-04-26 12:31:49,16,memory and file,9.427
5,2019-04-26 12:31:58,32,memory and file,10.253
6,2019-04-26 12:32:09,64,memory and file,10.16


In [27]:
names(tbl)

4-element Array{Symbol,1}:
 :datetime           
 Symbol("block size")
 :comment            
 :time               

# Add missing columns

In [28]:
size(tbl)

(6, 4)

In [53]:
function prepare_empty_col(sz, tp=Any)
    arr = Array{tp, 1}(undef, sz)
    for i in 1:sz
        arr[i] = "NA"
#         arr[i] = nothing
    end
    return arr
end

function add_empty_col!(tbl, col_name)
#     col = prepare_empty_col(size(tbl, 1), String)
    col = prepare_empty_col(size(tbl, 1))
#     println(col)
    if typeof(col_name) == Symbol
        column_name = col_name
    else
        column_name = Symbol(col_name)
    end
#     print(column_name)
#     tbl[column_name] = 1:size(tbl,1)
    tbl[column_name] = col
#     display(tbl)
    return tbl
    
end

add_empty_col! (generic function with 1 method)

In [30]:
# add_empty_col!(tbl, "pok u s")

In [31]:
function add_missing_cols!(tbl, to_add)
        
    nms = [String(name) for name in names(tbl)]


    for key in keys(to_add)

        if key in nms
#             println("key '$key' exists")
        else
            add_empty_col!(tbl, key)
        end

    end
    return tbl
end

add_missing_cols! (generic function with 1 method)

In [32]:
add_missing_cols!(tbl, to_add)

,datetime,block size,comment,time,new0
,String,Int64,String,Float64,Any
1,2019-04-26 12:26:10,16,computed,126.29,
2,2019-04-26 12:28:16,32,computed,111.373,
3,2019-04-26 12:30:08,64,computed,101.213,
4,2019-04-26 12:31:49,16,memory and file,9.427,
5,2019-04-26 12:31:58,32,memory and file,10.253,
6,2019-04-26 12:32:09,64,memory and file,10.16,


# Add row

In [46]:

# nms = [String(name) for name in names(tbl)]
function prepare_empty_row(tbl)
    new_row = Dict()
    for nm in names(tbl)
        if eltype(tbl[nm]) <: Number
#             println("$nm is subtype Number")
            new_row[nm] = 0
        elseif eltype(tbl[nm]) == String
#             println("$nm is String")
            new_row[nm] = ""
        else
            new_row[nm] = nothing

        end
    end
        
    return new_row
end

prepare_empty_row (generic function with 1 method)

In [34]:
new_row = prepare_empty_row(tbl)

datetime is String
block size is subtype Number
comment is String
time is subtype Number


Dict{Any,Any} with 5 entries:
  :time                => 0
  :new0                => nothing
  :datetime            => ""
  :comment             => ""
  Symbol("block size") => 0

In [35]:
function dict_string_keys_to_symbols(to_add)
    new_to_add = Dict()
    for (key, val) in to_add
    #     println(key)
        new_to_add[Symbol(key)] = val
    end
    return new_to_add
end

dict_string_keys_to_symbols (generic function with 1 method)

In [36]:
new_to_add = dict_string_keys_to_symbols(to_add)
merge!(new_row, new_to_add)
push!(tbl, new_row)

,datetime,block size,comment,time,new0
,String,Int64,String,Float64,Any
1,2019-04-26 12:26:10,16,computed,126.29,
2,2019-04-26 12:28:16,32,computed,111.373,
3,2019-04-26 12:30:08,64,computed,101.213,
4,2019-04-26 12:31:49,16,memory and file,9.427,
5,2019-04-26 12:31:58,32,memory and file,10.253,
6,2019-04-26 12:32:09,64,memory and file,10.16,
7,,6,,0.0,new


In [37]:
tbl

,datetime,block size,comment,time,new0
,String,Int64,String,Float64,Any
1,2019-04-26 12:26:10,16,computed,126.29,
2,2019-04-26 12:28:16,32,computed,111.373,
3,2019-04-26 12:30:08,64,computed,101.213,
4,2019-04-26 12:31:49,16,memory and file,9.427,
5,2019-04-26 12:31:58,32,memory and file,10.253,
6,2019-04-26 12:32:09,64,memory and file,10.16,
7,,6,,0.0,new


In [38]:
push!(tbl, Dict(:datetime => "454", Symbol("block size") => 6, :comment => "", :time => 0.0, Symbol("new0") => "i"))

,datetime,block size,comment,time,new0
,String,Int64,String,Float64,Any
1,2019-04-26 12:26:10,16,computed,126.29,
2,2019-04-26 12:28:16,32,computed,111.373,
3,2019-04-26 12:30:08,64,computed,101.213,
4,2019-04-26 12:31:49,16,memory and file,9.427,
5,2019-04-26 12:31:58,32,memory and file,10.253,
6,2019-04-26 12:32:09,64,memory and file,10.16,
7,,6,,0.0,new
8,454,6,,0.0,i


In [39]:
eltype(tbl.datetime)<:Number

false

In [40]:
# tbl[:new3] = [1,2,3,4,5,6]

In [41]:
# tbl[Symbol("s sa dskj")] = 1:6

In [42]:
tbl

,datetime,block size,comment,time,new0
,String,Int64,String,Float64,Any
1,2019-04-26 12:26:10,16,computed,126.29,
2,2019-04-26 12:28:16,32,computed,111.373,
3,2019-04-26 12:30:08,64,computed,101.213,
4,2019-04-26 12:31:49,16,memory and file,9.427,
5,2019-04-26 12:31:58,32,memory and file,10.253,
6,2019-04-26 12:32:09,64,memory and file,10.16,
7,,6,,0.0,new
8,454,6,,0.0,i


In [43]:
df = DataFrame(A = Int[], B = String[])
push!(df, [2, "N"])
push!(df, Dict(:B => "F", :A => 3))

,A,B
,Int64,String
1,2,N
2,3,F


# All in one piece

In [56]:
to_add = Dict()
to_add["block size"] = 6
to_add["new0"] = "new"

tbl = CSV.read("../data2.csv")
display(typeof(tbl))
display(tbl)

DataFrame

,datetime,block size,comment,time
,String,Int64,String,Float64
1,2019-04-26 12:26:10,16,computed,126.29
2,2019-04-26 12:28:16,32,computed,111.373
3,2019-04-26 12:30:08,64,computed,101.213
4,2019-04-26 12:31:49,16,memory and file,9.427
5,2019-04-26 12:31:58,32,memory and file,10.253
6,2019-04-26 12:32:09,64,memory and file,10.16


In [57]:
tbl = DataFrame(tbl)
add_missing_cols!(tbl, to_add)
new_row = prepare_empty_row(tbl)
new_to_add = dict_string_keys_to_symbols(to_add)
merge!(new_row, new_to_add)
push!(tbl, new_row)
display(tbl)
CSV.write("new_data.csv", tbl)

,datetime,block size,comment,time,new0
,String,Int64,String,Float64,Any
1,2019-04-26 12:26:10,16,computed,126.29,NA
2,2019-04-26 12:28:16,32,computed,111.373,NA
3,2019-04-26 12:30:08,64,computed,101.213,NA
4,2019-04-26 12:31:49,16,memory and file,9.427,NA
5,2019-04-26 12:31:58,32,memory and file,10.253,NA
6,2019-04-26 12:32:09,64,memory and file,10.16,NA
7,,6,,0.0,new


"new_data.csv"